In [10]:
'''
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.
'''

'\nEscreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.\n'

# TODO
# Guardar Título e Journal dos papers
# Guardar Ref_Context

In [39]:
'''
prompt:
Gere código python que cria uma classe Reference que tem atributos: context, page, raw
'''

class Reference:
    def __init__(self, context: str, page: int, raw: str):
        """
        Inicializa um objeto Reference.

        Parâmetros:
            context (str): Texto ao redor da referência (ex: sentença completa).
            page (int): Número da página onde a referência foi encontrada.
            raw (str): Texto cru da referência (por exemplo, conteúdo da tag <ref>).
        """
        self.context = context
        self.page = page
        self.raw = raw

    def __repr__(self):
        return f"Reference(page={self.page}, raw='{self.raw[:30]}...', context='{self.context[:30]}...')"


In [ ]:
'''
prompt:
Write a function parse_tei(tei_filepath) that opens the TEI XML file and counts the number of biblStruct entries. 
Also, count the number of <s></s> entries. 
Return three values: the number of s entries, the number of biblStruct entries and a list of Reference objects. 
Each reference object has a field raw and a field context. Raw should be filled with the text inside the ref tag; context should be filled with the text on the parent <s> tag.
'''

import xml.etree.ElementTree as ET

def parse_tei(tei_filepath):
    """
    Parse a TEI XML file and return:
    - the number of <s> elements
    - the number of <biblStruct> elements
    - a list of Reference objects with raw and context

    Returns:
        (int, int, list of Reference)
    """
    try:
        tree = ET.parse(tei_filepath)
        root = tree.getroot()

        # Handle TEI namespaces if present
        ns = {'tei': root.tag.split('}')[0].strip('{')} if '}' in root.tag else {}

        # Count <s> elements
        s_elements = root.findall('.//tei:s', ns) if ns else root.findall('.//s')
        num_s = len(s_elements)

        # Count <biblStruct> elements
        bibl_structs = root.findall('.//tei:biblStruct', ns) if ns else root.findall('.//biblStruct')
        num_bibl_structs = len(bibl_structs)

        # Extract <ref> text and corresponding <s> context
        references = []
        for s in s_elements:
            # Get the full text of <s> (its string content)
            context = ''.join(s.itertext()).strip()

            # Find all <ref> elements inside this <s>
            ref_elements = s.findall('.//tei:ref', ns) if ns else s.findall('.//ref')
            for ref in ref_elements:
                raw = ref.text.strip() if ref.text else ""
                references.append(Reference(raw=raw, context=context, page=1))

        return num_s, num_bibl_structs, references

    except ET.ParseError as e:
        print(f"Error parsing XML '{tei_filepath}': {e}")
        return 0, 0, []
    except FileNotFoundError:
        print(f"File not found: {tei_filepath}")
        return 0, 0, []

paper_path = "../data/teis/from-scopus/A-Historical-Intervention-in-the-Opportunity-Wars-Forgotten-Scholarship-the-DiscoveryCreation-Disruption-and-Moving-Forward-by-Looking-Backward_2023_SAGE-Publications-Ltd.pdf.grobid.tei.xml"

sentence_count, reference_count, refs = parse_tei(paper_path) 
assert sentence_count == 283
assert reference_count == 106

assert len(refs) == 222



222

In [ ]:
'''
prompt:
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e cria dois dataframes.
Um pandas df chamado "papers_df" em que cada registro tem a coluna paper_id, que é um número sequencial. 
Para cada arquivo, invoque parse_tei(tei_filepath), que retorna sentence_count, reference_count e refs.  
Adicione sentence_count e reference_count no papers_df.  
Ao outro df, chamado refs_df, adicione todos os refs. Um ref é um objeto Reference que tem os campos raw, context e page. Cada campo deve ser uma coluna em refs_df.
'''

import os
import pandas as pd

def read_tei_papers(path):
    """
    Lê arquivos TEI XML de um diretório e retorna:
    - papers_df: DataFrame com paper_id, filename, sentence_count, reference_count
    - refs_df: DataFrame com paper_id, raw, context, page
    """
    xml_files = [f for f in os.listdir(path) if f.lower().endswith('.xml')]

    papers_data = []
    refs_data = []

    for i, filename in enumerate(xml_files):
        paper_id = i + 1
        filepath = os.path.join(path, filename)

        sentence_count, reference_count, refs = parse_tei(filepath)

        papers_data.append({
            'paper_id': paper_id,
            'filename': filename,
            'sentence_count': sentence_count,
            'reference_count': reference_count
        })

        for ref in refs:
            refs_data.append({
                'paper_id': paper_id,
                'raw': ref.raw,
                'context': ref.context,
                'page': ref.page
            })

    papers_df = pd.DataFrame(papers_data)
    refs_df = pd.DataFrame(refs_data)

    return papers_df, refs_df

In [55]:
papers_df, refs_df = read_tei_papers("../data/teis/from-scopus")

papers_df.sort_values(by='reference_count', ascending=False)

papers_df.to_csv("../data/papers.csv")
refs_df.to_csv("../data/refs.csv")

Error parsing XML '../data/teis/from-scopus\Property-is-only-another-name-for-decentralized-creation-of-knowledge_2019_Springer-New-York-LLC-barbarabbertramgskcom.pdf.grobid.tei.xml': no element found: line 1, column 0
